ação.
API: https://www.sptrans.com.br/desenvolvedores
ENDPOINTS:https://www.sptrans.com.br/desenvolvedores/api-do-olho-vivo-guia-de-r
eferencia/documentacao-api/
Utilizar o GTFS da SPTRANS para dados complementares (Dados
Estáticos/Cadastrais).
Enriquecer os dados de paradas com o endereço da localidade, através de latitude e
longitude.
Dados do GTFS: https://gtfs.org/documentation/schedule/reference/


In [92]:
import geopy.distance



In [93]:
%%timeit
coords_1 = (52.2296756, 21.0122287)# (lat, lon)
coords_2 = (52.406374, 16.9251681) # (lat, lon)

geopy.distance.geodesic(coords_1, coords_2).km

# The latitude of Sao Paulo City, Brazil is -23.533773, and the longitude is -46.625290.

39.2 μs ± 358 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [94]:
import collections

import datetime
previsao = result.json()

def convert_time(text):
    h,m = text.split(':')
    return datetime.datetime(2025, 1, 17, int(h), int(m), 0)

hora_atual = convert_time(previsao.get('hr'))
print(hora_atual)

onibus = collections.defaultdict(list)
for ps in previsao.get('ps',[]):
    cod_parada = ps.get('cp','?')
    ps_px = ps.get('px') # long
    ps_py = ps.get('py') # lat 
    vs = ps.get('vs',[])
    for veiculo in vs:
        vs_p = veiculo.get('p')
        vs_t = veiculo.get('t')
        vs_px = veiculo.get('px')
        vs_py = veiculo.get('py')
        print(vs_p)
        onibus[vs_p].append((cod_parada, vs_t, round(geopy.distance.geodesic((vs_py, vs_px), (ps_py, ps_px)).km,ndigits=3)))
        
    

onibus    

NameError: name 'result' is not defined

In [98]:
import openlayer


In [99]:
# !pip install ipyleaflet jupyter-leaflet
# conda install conda-forge::nodejs
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-leaflet
import ipyleaflet

In [100]:
from ipyleaflet import Map, Marker

center = (-23.5558, -46.6396)

m = Map(center=center, zoom=10)
marker = Marker(location=center, draggable=True)
m.add(marker);

In [103]:
import geopy.distance

coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print(geopy.distance.geodesic(coords_1, coords_2).km)

279.35290160430094


In [104]:
import pandas as pd

BASE_PATH = 'trafi_gtfs'

df_agency = pd.read_csv(f'{BASE_PATH}/agency.txt', header=0)
df_agency.head()

,agency_id,agency_name,agency_url,agency_timezone,agency_lang
0,1,SPTRANS,http://www.sptrans.com.br/?versao=200125,America/Sao_Paulo,pt


In [105]:
df_calendar = pd.read_csv(f'{BASE_PATH}/calendar.txt', header=0)
df_calendar.head()

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,USD,1,1,1,1,1,1,1,20231001,20250801
1,U__,1,1,1,1,1,0,0,20231001,20250801
2,US_,1,1,1,1,1,1,0,20231001,20250801
3,_SD,0,0,0,0,0,1,1,20231001,20250801
4,__D,0,0,0,0,0,0,1,20231001,20250801


In [106]:
df_fare_attributes = pd.read_csv(f'{BASE_PATH}/fare_attributes.txt', header=0)
df_fare_attributes.head()

,fare_id,price,currency_type,payment_method,transfers,transfer_duration
0,CPTM,5.2,BRL,0,NaN,10800
1,Metrô,5.2,BRL,0,NaN,10800
2,Ônibus,5.0,BRL,0,NaN,10800
3,Ônibus + CPTM,8.9,BRL,0,NaN,10800
4,Ônibus + Metrô,8.9,BRL,0,NaN,10800


In [107]:
df_fare_rules = pd.read_csv(f'{BASE_PATH}/fare_rules.txt', header=0)
print(df_fare_rules.shape)
df_fare_rules.head()


(5355, 5)


,fare_id,route_id,origin_id,destination_id,contains_id
0,CPTM,CPTM L07,NaN,NaN,NaN
1,CPTM,CPTM L08,NaN,NaN,NaN
2,CPTM,CPTM L09,NaN,NaN,NaN
3,CPTM,CPTM L10,NaN,NaN,NaN
4,CPTM,CPTM L11,NaN,NaN,NaN


In [108]:
df_frequencies = pd.read_csv(f'{BASE_PATH}/frequencies.txt', header=0)
print(df_frequencies.shape)
df_frequencies.head()

(40066, 4)


,trip_id,start_time,end_time,headway_secs
0,1012-10-0,00:00:00,00:59:00,1200
1,1012-10-0,01:00:00,01:59:00,3600
2,1012-10-0,05:00:00,05:59:00,600
3,1012-10-0,06:00:00,06:59:00,600
4,1012-10-0,07:00:00,07:59:00,600


In [109]:
df_routes = pd.read_csv(f'{BASE_PATH}/routes.txt', header=0)
print(df_routes.shape)

df_routes.head()

(1342, 7)


,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
0,1012-10,1,1012-10,Term. Jd. Britania - Jd. Monte Belo,3,509E2F,FFFFFF
1,1012-21,1,1012-21,Term. Jd. Britânia - Jd. Rosinha,3,509E2F,FFFFFF
2,1014-10,1,1014-10,Morro Doce - Cptm Perus,3,509E2F,FFFFFF
3,1015-10,1,1015-10,Term. Jd. Britânia - Chác. Maria Trindade,3,509E2F,FFFFFF
4,1016-10,1,1016-10,Cem. Do Horto - Shop. Center Norte,3,002F6C,FFFFFF


# shapes

Primary key (shape_id, shape_pt_sequence)

Shapes describe the path that a vehicle travels along a route alignment, and are defined in the file shapes.txt. Shapes are associated with Trips, and consist of a sequence of points through which the vehicle passes in order. Shapes do not need to intercept the location of Stops exactly, but all Stops on a trip should lie within a small distance of the shape for that trip, i.e. close to straight line segments connecting the shape points. The shapes.txt file should be included for all route-based services (not for zone-based demand-responsive services).

In [110]:
df_shapes = pd.read_csv(f'{BASE_PATH}/shapes.txt', header=0)
df_shapes.describe()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
count,1.128578e+06,1.128578e+06,1.128578e+06,1.128578e+06,1.128578e+06
mean,7.446976e+04,-2.357536e+01,-4.662821e+01,3.034500e+02,9.268232e+03
std,9.928185e+03,8.675968e-02,1.082632e-01,2.270077e+02,7.263211e+03
min,1.783800e+04,-2.391133e+01,-4.698404e+01,1.000000e+00,0.000000e+00
25%,7.016700e+04,-2.363112e+01,-4.671163e+01,1.270000e+02,3.574785e+03
50%,7.914300e+04,-2.356345e+01,-4.665345e+01,2.600000e+02,7.760304e+03
75%,8.223000e+04,-2.351169e+01,-4.654665e+01,4.310000e+02,1.342220e+04
max,8.367200e+04,-2.319474e+01,-4.618493e+01,1.824000e+03,6.062177e+04


In [111]:
from ipyleaflet import Map, Marker, FullScreenControl

shape_pos = df_shapes[df_shapes['shape_id'] == 81072][['shape_pt_lat', 'shape_pt_lon']].values 

avg_lat = sum(x[0] for x in shape_pos)/len(shape_pos)
avg_long = sum(x[1] for x in shape_pos)/len(shape_pos)

control = FullScreenControl()

# center = (-23.5558, -46.6396)
center = (avg_lat, avg_long)


m1 = Map(center=center, zoom=12)
m1.add_control(control)

for p in shape_pos:
    marker = Marker(location=tuple(p), draggable=False)
    m1.add(marker);

display(m1)

Map(center=[-23.436078552542355, -46.79888413220338], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [112]:
df_stop_times = pd.read_csv(f'{BASE_PATH}/stop_times.txt', header=0)
df_stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,1012-10-0,18:00:00,18:00:00,301790,1
1,1012-10-0,18:01:35,18:01:35,301764,2
2,1012-10-0,18:03:10,18:03:10,301724,3
3,1012-10-0,18:04:45,18:04:45,301730,4
4,1012-10-0,18:06:20,18:06:20,30003042,5


In [113]:
df_stops = pd.read_csv(f'{BASE_PATH}/stops.txt', header=0)
print(df_stops.shape)
df_stops.head()


(21842, 5)


,stop_id,stop_name,stop_desc,stop_lat,stop_lon
0,18848,Clínicas,NaN,-23.554022,-46.671108
1,18849,Vila Madalena,NaN,-23.546498,-46.691141
2,18850,Consolação,NaN,-23.558094,-46.660205
3,18851,Conceição,NaN,-23.635039,-46.641239
4,18852,Jabaquara,NaN,-23.646033,-46.641028


In [114]:
from ipyleaflet import Map, Marker, FullScreenControl
shape_pos = df_stops[['stop_lat', 'stop_lon']].values[:500] 

avg_lat = sum(x[0] for x in shape_pos)/len(shape_pos)
avg_long = sum(x[1] for x in shape_pos)/len(shape_pos)

control = FullScreenControl()

# center = (-23.5558, -46.6396)
center = (avg_lat, avg_long)


m1 = Map(center=center, zoom=12)
m1.add_control(control)

for p in shape_pos:
    marker = Marker(location=tuple(p), draggable=False)
    m1.add(marker);

display(m1)

Map(center=[-23.524372034, -46.63580856000003], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zo…

In [115]:
# https://gtfs.org/documentation/schedule/reference/#tripstxt

df_trips = pd.read_csv(f'{BASE_PATH}/trips.txt', header=0)
df_trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id
0,1012-10,USD,1012-10-0,Jd. Monte Belo,0,81072
1,1012-10,USD,1012-10-1,Term. Jd. Britania,1,81073
2,1012-21,U__,1012-21-0,Jd. Rosinha,0,81195
3,1014-10,U__,1014-10-0,Cptm Perus,0,82855
4,1015-10,USD,1015-10-0,Chác. Maria Trindade,0,81148


trips (route_id -> Shape_id)
df_routes (-)
stops (-)


In [119]:
df_stops

,stop_id,stop_name,stop_desc,stop_lat,stop_lon
0,18848,Clínicas,NaN,-23.554022,-46.671108
1,18849,Vila Madalena,NaN,-23.546498,-46.691141
2,18850,Consolação,NaN,-23.558094,-46.660205
3,18851,Conceição,NaN,-23.635039,-46.641239
4,18852,Jabaquara,NaN,-23.646033,-46.641028
...,...,...,...,...,...
21837,960011507,"R. Caminho Do Engenho, 255",Ref.: R Bartolo Di Fredi/ R Padre Antonio Link,-23.595972,-46.741941
21838,960011508,"R. Caminho Do Engenho, 417",Ref.: R Bartolomeu Bandinelli/ R Bom Jesus Dos...,-23.596402,-46.743436
21839,960011509,"Av. Dr. Guilherme Dumont Villares, 50",Ref.: R Alfredo Mendes Da Silva/ Av Professor ...,-23.604681,-46.747021
21840,960011510,"R. Alfredo Mendes Da Silva, 499",Ref.: R Cedro/ Av Doutor Guilherme Dumont Vill...,-23.604420,-46.748357


In [120]:
df_shapes


,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,81072,-23.432024,-46.787121,1,0.000000
1,81072,-23.431827,-46.787153,2,22.061337
2,81072,-23.431764,-46.787163,3,29.113285
3,81072,-23.431614,-46.787187,4,45.906006
4,81072,-23.431590,-46.787191,5,48.596806
...,...,...,...,...,...
1128573,62557,-23.654782,-46.761548,1077,24199.273000
1128574,62557,-23.654284,-46.762232,1078,24288.221000
1128575,62557,-23.654177,-46.762379,1079,24307.334000
1128576,62557,-23.654077,-46.762517,1080,24325.246000


In [121]:
def get_shape_from_id(shape_id):
    return df_shapes[df_shapes['shape_id'] == shape_id][['shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence']].values

get_shape_from_id(62557)

array([[-2.3654077e+01, -4.6762517e+01,  1.0000000e+00],
       [-2.3653972e+01, -4.6762661e+01,  2.0000000e+00],
       [-2.3653877e+01, -4.6762792e+01,  3.0000000e+00],
       ...,
       [-2.3654177e+01, -4.6762379e+01,  1.0790000e+03],
       [-2.3654077e+01, -4.6762517e+01,  1.0800000e+03],
       [-2.3653972e+01, -4.6762661e+01,  1.0810000e+03]])

In [122]:
def get_trips_stop_seq(trip_id):
    trip_seq = df_stop_times[df_stop_times['trip_id'] == trip_id][['stop_id','stop_sequence']].values
    result = [ (i,tuple(df_stops[df_stops['stop_id'] == s].values[0])) for s,i in trip_seq ]
    return result
get_trips_stop_seq('1012-10-0')

[(1, (301790, 'Terminal Jardim Britânia', nan, -23.431924, -46.787131)),
 (2,
  (301764,
   'R. Delsuc Alves De Magalhães, 194',
   'Ref.: R. Diego Velasquez',
   -23.435427,
   -46.78846)),
 (3,
  (301724,
   'R. Delsuc Alves De Magalhães, 538',
   'Ref.: Estr. Cel. José Gladiador',
   -23.435469,
   -46.791522)),
 (4,
  (301730,
   'Estr. Cel. José Gladiador, 262',
   'Ref.: R. Delsuc Alves De Magalhães',
   -23.435386,
   -46.791885)),
 (5,
  (30003042,
   'R. Virgínia Castiglioni, 381',
   'Ref.: R Salvador Albano/ R Virginia Castiglioni',
   -23.433008,
   -46.792717)),
 (6,
  (30003045,
   'R. Nestor Gomes',
   'Ref.: R Tito Batista/ R Virginia Castiglioni',
   -23.431541,
   -46.792973)),
 (7,
  (30003044,
   'R. Alberto Alvarez, 135',
   'Ref.: Rua Dionisio Bellante',
   -23.430041,
   -46.794827)),
 (8,
  (30003043,
   'R. Dionísio Bellante, 29',
   'Ref.: R Guilherme Correa/ R Leopoldo De Passos Lima',
   -23.4285,
   -46.795379)),
 (9,
  (30003013,
   'R. João Fernandes Viei

In [123]:
trip_seq = get_trips_stop_seq('1012-10-0')
trip_seq

[(1, (301790, 'Terminal Jardim Britânia', nan, -23.431924, -46.787131)),
 (2,
  (301764,
   'R. Delsuc Alves De Magalhães, 194',
   'Ref.: R. Diego Velasquez',
   -23.435427,
   -46.78846)),
 (3,
  (301724,
   'R. Delsuc Alves De Magalhães, 538',
   'Ref.: Estr. Cel. José Gladiador',
   -23.435469,
   -46.791522)),
 (4,
  (301730,
   'Estr. Cel. José Gladiador, 262',
   'Ref.: R. Delsuc Alves De Magalhães',
   -23.435386,
   -46.791885)),
 (5,
  (30003042,
   'R. Virgínia Castiglioni, 381',
   'Ref.: R Salvador Albano/ R Virginia Castiglioni',
   -23.433008,
   -46.792717)),
 (6,
  (30003045,
   'R. Nestor Gomes',
   'Ref.: R Tito Batista/ R Virginia Castiglioni',
   -23.431541,
   -46.792973)),
 (7,
  (30003044,
   'R. Alberto Alvarez, 135',
   'Ref.: Rua Dionisio Bellante',
   -23.430041,
   -46.794827)),
 (8,
  (30003043,
   'R. Dionísio Bellante, 29',
   'Ref.: R Guilherme Correa/ R Leopoldo De Passos Lima',
   -23.4285,
   -46.795379)),
 (9,
  (30003013,
   'R. João Fernandes Viei

In [124]:
df_trips[df_trips['route_id'] == '1012-10']

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id
0,1012-10,USD,1012-10-0,Jd. Monte Belo,0,81072
1,1012-10,USD,1012-10-1,Term. Jd. Britania,1,81073


In [125]:
df_routes[df_routes['route_id'] == '1012-10']

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
0,1012-10,1,1012-10,Term. Jd. Britania - Jd. Monte Belo,3,509E2F,FFFFFF


In [126]:
route_id = '1012-10'

route_data = df_routes[df_routes['route_id'] == route_id]
print(f'Rota ID = {route_id}')
display(route_data)
trips = df_trips[df_trips['route_id'] == route_id]
print('Contem as trips:')
display(trips)

Rota ID = 1012-10


,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
0,1012-10,1,1012-10,Term. Jd. Britania - Jd. Monte Belo,3,509E2F,FFFFFF


Contem as trips:


,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id
0,1012-10,USD,1012-10-0,Jd. Monte Belo,0,81072
1,1012-10,USD,1012-10-1,Term. Jd. Britania,1,81073


In [127]:
trip_seq = get_trips_stop_seq('1012-10-0')
trip_seq

[(1, (301790, 'Terminal Jardim Britânia', nan, -23.431924, -46.787131)),
 (2,
  (301764,
   'R. Delsuc Alves De Magalhães, 194',
   'Ref.: R. Diego Velasquez',
   -23.435427,
   -46.78846)),
 (3,
  (301724,
   'R. Delsuc Alves De Magalhães, 538',
   'Ref.: Estr. Cel. José Gladiador',
   -23.435469,
   -46.791522)),
 (4,
  (301730,
   'Estr. Cel. José Gladiador, 262',
   'Ref.: R. Delsuc Alves De Magalhães',
   -23.435386,
   -46.791885)),
 (5,
  (30003042,
   'R. Virgínia Castiglioni, 381',
   'Ref.: R Salvador Albano/ R Virginia Castiglioni',
   -23.433008,
   -46.792717)),
 (6,
  (30003045,
   'R. Nestor Gomes',
   'Ref.: R Tito Batista/ R Virginia Castiglioni',
   -23.431541,
   -46.792973)),
 (7,
  (30003044,
   'R. Alberto Alvarez, 135',
   'Ref.: Rua Dionisio Bellante',
   -23.430041,
   -46.794827)),
 (8,
  (30003043,
   'R. Dionísio Bellante, 29',
   'Ref.: R Guilherme Correa/ R Leopoldo De Passos Lima',
   -23.4285,
   -46.795379)),
 (9,
  (30003013,
   'R. João Fernandes Viei

In [128]:
# get_shape_from_id(81073)


In [138]:
list_pos_exemplo = get_shape_from_id(81073)
lat_exemplo = -23.4285
long_exemplo = -46.795379

In [146]:

def find_closest(list_pos, s_lat, s_long):
    # print(s_lat, s_long)
    best_pos_index = None
    best_dist = 999999
    for lat, long, shape_pt_sequence in list_pos:
        # print(lat, long, shape_pt_sequence)
        dist = geopy.distance.geodesic((s_lat, s_long), (lat, long)).meters
        if dist < best_dist:
            best_dist = dist
            best_pos_index = shape_pt_sequence
    return best_pos_index, best_dist

    

find_closest(list_pos_exemplo, lat_exemplo, long_exemplo)


(218.0, 32.451481198181476)

In [147]:
%%timeit
find_closest(list_pos_exemplo, lat_exemplo, long_exemplo)


9.37 ms ± 40.4 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [135]:
list_pos_exemplo[83]

array([-23.428219, -46.795469,  84.      ])

In [137]:
%time
def conciliar_trip_stop(trip_id, shape_id):
    trip_seq = get_trips_stop_seq(trip_id)
    shape = get_shape_from_id(shape_id)
    index = {}
    for i, stop in trip_seq:
        closest = find_closest(shape, stop[3], stop[4])
        # print(closest) # shape_pt_sequence, distance
        index[closest[0]] = (stop[0], closest[1])

    shape_for_trip = df_shapes[df_shapes['shape_id'] == shape_id].copy() 
    shape_for_trip['stop_id'] = [index.get(x, (None, None))[0] for x in shape_for_trip['shape_pt_sequence'].values]
    shape_for_trip['stop_distance'] = [index.get(x, (None, None))[1] for x in shape_for_trip['shape_pt_sequence'].values]
    return shape_for_trip
                     
conciliar_trip_stop('1012-10-0', 81072)

CPU times: user 3 μs, sys: 1 μs, total: 4 μs
Wall time: 6.91 μs


,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,stop_id,stop_distance
0,81072,-23.432024,-46.787121,1,0.000000,NaN,NaN
1,81072,-23.431827,-46.787153,2,22.061337,301790.0,10.975496
2,81072,-23.431764,-46.787163,3,29.113285,NaN,NaN
3,81072,-23.431614,-46.787187,4,45.906006,NaN,NaN
4,81072,-23.431590,-46.787191,5,48.596806,NaN,NaN
...,...,...,...,...,...,...,...
290,81072,-23.444853,-46.799501,291,6890.008800,NaN,NaN
291,81072,-23.444664,-46.799499,292,6910.941400,NaN,NaN
292,81072,-23.444473,-46.799500,293,6939.022900,NaN,NaN
293,81072,-23.444412,-46.799515,294,6946.085900,301771.0,4.520668


In [ ]:
d = df_stops[df_stops['stop_id'] == 301790]
lat = d.values[0][3]
long = d.values[0][4]
print(lat, long)
display(d)
geopy.distance.geodesic((-23.431827, -46.787153), (lat, long)).meters

## Ideias:

* Tempo de atraso nos pontos (stops)
* Velocidade media
* Deteccao de eventos/emergencias


## Tasks: 

- Subir MiniIO
- manter um sistema gravando no MiniOI dados de posicao
- sistema para processar os dados - Pyspark

# docker composer
- MiniIO
- PySpark
- Trino
- Metabase / SuperSet
- 


## Questoes
- Todas as posicoes? Ou só algumas linhas?
- frequencia de atualizacao (1 minuto)


previsão da linha
-> 


acertividade da previsão
-> differenca entre o previsto e o realizado



-> pega a posicao dos onibus usando a API (horario de referencia, prefixo veiculo, linha, etc)


-> shapes -> unir shapes com stops (percuso + paradas)
